# Soccer Hours


## Setting up Database using Python

In [395]:
# Import sqlite3 module
import sqlite3
import getpass
import datetime

# Connect to the soccerhours database
db=sqlite3.connect("/Users/subhayuchakravarty/AGIT/Github/soccerhours/soccerhours/soccerhours.db")
cursor = db.cursor()


In [255]:
# Create table Users - Stores User data
cursor.execute('''
CREATE TABLE user (
    pk_user_id INTEGER PRIMARY KEY AUTOINCREMENT,
    user_fname VARCHAR,
    user_lname VARCHAR,
    username VARCHAR NOT NULL,
    admin_status VARCHAR NOT NULL,
    user_status VARCHAR NOT NULL,
    user_email VARCHAR NOT NULL,
    user_password VARCHAR NOT NULL,
    zip_code INTEGER NOT NULL,
    date_added TIMESTAMP NOT NULL
)
''')


In [289]:
cursor.execute(''' Insert into sqlite_sequence values(\'user\',100) ''')
cursor.execute(''' Insert into sqlite_sequence values(\'venue\',200) ''')
cursor.execute(''' Insert into sqlite_sequence values(\'events\',300) ''')
db.commit()

In [259]:
# Create table Venues - Stores Venue data
cursor.execute('''
CREATE TABLE venue (
    pk_venue_id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
    venue_name VARCHAR NOT NULL,
    venue_desc VARCHAR NOT NULL,
    venue_st_addr VARCHAR NOT NULL,
    venue_zip_code INTEGER NOT NULL
)
''')

In [260]:
# Create table Venues - Stores Venue data
cursor.execute('''
CREATE TABLE events (
    pk_event_id INTEGER PRIMARY KEY AUTOINCREMENT,
    event_name VARCHAR NOT NULL,
    event_desc VARCHAR NOT NULL,
    event_date DATETIME NOT NULL,
    start_time INTEGER NOT NULL,
    end_time INTEGER NOT NULL,
    user_id INTEGER NOT NULL,    
    venue_id INTEGER NOT NULL,
    event_capacity INTEGER NOT NULL,
    event_status VARCHAR NOT NULL,
    gender_option VARCHAR NOT NULL,
    members_joined INTEGER NOT NULL,
    slots VARCHAR NOT NULL
)
''')

In [261]:
# Create table Venues - Stores Venue data
cursor.execute('''
CREATE TABLE slots (
    slot_id INTEGER NOT NULL,
    venue_id INTEGER NOT NULL,
    date DATETIME NOT NULL,
    availability VARCHAR NOT NULL
)
''')

In [262]:
# Create table Venues - Stores Venue data
cursor.execute('''
CREATE TABLE event_members (
    event_id INTEGER NOT NULL,
    user_id INTEGER NOT NULL
)
''')

In [303]:
# Create Static table for Slot ID defintions
cursor.execute('''
CREATE TABLE timeslots (
    start_time INTEGER NOT NULL,
    slot_id INTEGER NOT NULL
)
''')

db.commit()

OperationalError: table timeslots already exists

In [304]:
cursor.execute('''
INSERT INTO timeslots VALUES (8,1),(9,2),(10,3),(11,4),(12,5),(13,6),(14,7),(15,8),(16,9),(17,10),(18,11),(19,12),(20,13);
''')

db.commit()

### Common Functions

In [291]:
# Duplicate UserName check
def dup_username_check(username):

    cursor.execute(''' SELECT username from user ''')
    usernames=cursor.fetchall()
    if ((username,) in usernames):
        print("UserName {0} already exists.".format(username))
        return False
    else:
        print("Username available, please proceed.")
        return True

In [292]:
# Duplicate Email check
def dup_email_check(email):

    cursor.execute(''' SELECT user_email from user ''')
    emails=cursor.fetchall()
    if (email,) in emails:
        print("User with email address {0} already exists.".format(email))
        return False
    print("Unique email address, please proceed.")
    return True

In [297]:
# Duplicate Venue check
def dup_venue_check(venuename,zipcode):

    cursor.execute(''' SELECT venue_name, venue_zip_code from venue ''')
    venues=cursor.fetchall()
    venue=(venuename,zipcode)

    if (venue in venues):
        print("Venue with name: \'{0}\' and Zip code: {1} already exists.".format(venuename,zipcode))
        return False
    else:
        return True

In [376]:
# Function to get slot IDs from start and end times

def get_slot_ids(starttime, endtime):
    diff = endtime - starttime
    slotstarts=starttime
    slotids = list()
    cursor.execute(''' SELECT slot_id from timeslots where start_time= ? ''',(starttime,))
    slotid=cursor.fetchone()[0]
    slotids.append(int(slotid))
    
    if diff>1:
        for i in range(diff-1): 
            slotids.append(slotids[i]+1)
        return slotids    
    else:
        return slotids[0]

In [328]:
def check_valid_date(date):
    today=str(datetime.datetime.today()).split()[0]
    if (date>today):
        return 'G'
    elif(today>date):
        return 'L'
    else:
        return 'T'
    
    

In [342]:
def check_valid_time(starttime):
    now = datetime.datetime.now()
    if (now.hour >= int(starttime)):
        return False
    else:
        return True
    

In [355]:
def check_start_end_time(time,startorend):
    now = datetime.datetime.now()
    inttime=int(time)
    if (startorend=='S'):
        if (inttime<8 or inttime>20):
            return False
        else:
            return True
    else:
        if (inttime<8 or inttime>21):
            return False
        else:
            return True

In [334]:
def check_venue_slots(venueid, eventdate):
    cursor.execute(''' Select 1 from slots where venue_id=? and date=? ''',(venueid,eventdate))
    return bool(cursor.fetchone())
    

In [401]:
def check_valid_user(userid):
    cursor.execute(''' Select pk_user_id from user where pk_user_id=? ''',(userid,))
    user=cursor.fetchone()
    if(user!=None):
        return True
    return False
    

In [404]:
def check_valid_venue(venueid):
    cursor.execute(''' Select pk_venue_id from venue where pk_venue_id=? ''',(venueid,))
    venue=cursor.fetchone()
    if(venue!=None):
        return True
    return False

In [412]:
check_valid_event(301)

True

In [410]:
def check_valid_event(eventid):
    cursor.execute(''' Select pk_event_id from events where pk_event_id=? ''',(eventid,))
    event=cursor.fetchone()
    if(event!=None):
        return True
    return False

# Python Function Definitions

## 1. Add New User

Adds a user to the SoccerHours Database


In [290]:
# Adds One New user by inserting into table 'users'

def add_user(**userdata) :
    
    username=userdata['username']
    firstname=userdata['firstname']
    lastname=userdata['lastname']
    email=userdata['email']
    password=userdata['password']
    zipcode=userdata['zipcode']
    isadmin=userdata['isadmin']

    if isadmin=='Y':
        isadmin='A'
    else:
        isadmin='N'
    
    cursor.execute(''' INSERT INTO user VALUES((select seq from sqlite_sequence where name='user')+1,?,?,?,?,'A',?,?,?,?) ''',(firstname,lastname,username,isadmin,email,password,zipcode,datetime.datetime.now()))
    db.commit()


## Test Add_User

In [295]:
userdata=dict()
userdata['username']=input("UserName:")

s=dup_username_check(userdata['username'])
while(s==False):
    userdata['username']=input("Re-enter New UserName:")
    s=dup_username_check(userdata['username'])

userdata['firstname']=input("First Name:")
userdata['lastname']=input("Last Name:")
userdata['email']=input("Email ID:")
s=dup_email_check(userdata['email'])

while s==False:
    userdata['email']=input("New Email ID:")
    s=dup_email_check(userdata['email'])

userdata['password']=getpass.getpass("Password:")
cpassword=getpass.getpass("Confirm Password:")
while(cpassword!=userdata['password']):
    print("Passwords do not match")
    userdata['password']=getpass.getpass("Password:")
    cpassword=getpass.getpass("Confirm Password:")

userdata['zipcode']=input("Default Zip Code:")
userdata['isadmin']=input("Is this an Admin user? (Y/N): ")
add_user(**userdata)
    

UserName:k4rd4k
Username available, please proceed.
First Name:Abhishek
Last Name:Kardak
Email ID:akrdak@gmail.com
Unique email address, please proceed.
Password:········
Confirm Password:········
Default Zip Code:78751
Is this an Admin user? (Y/N): N


## 2. Add New Venue

Adds a user to the SoccerHours Database


In [296]:
# Adds One New Venue by inserting into table 'venue'

def add_venue(**venuedata) :
    
    venuename=venuedata['venuename']
    venuedesc=venuedata['venuedesc']
    venueaddr=venuedata['venueaddr']
    zipcode=venuedata['zipcode']
    
    # duplicate venue check

    # Inserts into venue
    cursor.execute(''' INSERT INTO venue VALUES((Select seq+1 from sqlite_sequence where name='venue'),?,?,?,?) ''',(venuename,venuedesc,venueaddr,zipcode))

    db.commit()


In [297]:
# Duplicate Venue check
def dup_venue_check(venuename,zipcode):

    cursor.execute(''' SELECT venue_name, venue_zip_code from venue ''')
    venues=cursor.fetchall()
    venue=(venuename,zipcode)

    if (venue in venues):
        print("Venue with name: \'{0}\' and Zip code: {1} already exists.".format(venuename,zipcode))
        return False
    else:
        return True

## Test Add_Venue

In [305]:
venuedata=dict()
venuedata['venuename']=input("Venue Short Name:")
venuedata['zipcode']=input("Zip Code:")

while (dup_venue_check(venuedata['venuename'], int(venuedata['zipcode'])) != True):
    venuedata['venuename']=input("New Venue's Short Name:")
    venuedata['zipcode']=input("New Venue's Zip Code:")

venuedata['venuedesc']=input("Venue Description:")
venuedata['venueaddr']=input("Address:")
add_venue(**venuedata)

Venue Short Name:Clark Field 2
Zip Code:78712
Venue Description:UT Clark field 2
Address:UT Austin


## 3. Start an Event

Adds an event into soccerhours database in Active status with 1 member added

In [359]:
# Adds the New Event by inserting into table 'events'

def start_event(**eventdata) :
    
    venueid=eventdata['venueid']    
    userid=eventdata['userid'] 
    eventname=eventdata['eventname']
    eventdesc=eventdata['eventdesc']
    eventdate=eventdata['eventdate']
    starttime=eventdata['starttime']
    endtime=eventdata['endtime']
    eventcapacity=eventdata['eventcapacity']
    genderoption=eventdata['genderoption']
    if(check_start_end_time(starttime,'S') and check_start_end_time(endtime,'E')):
        # Derive slots from StartTime, EndTime
        slotids=get_slot_ids(int(starttime),int(endtime))
    else:
        print("\nInvalid timings entered. Venue not booked.\n\nValid values are : \nRange of 8 - 20 for Start time\nRange of 9 - 21 for End time")
        return
    
    if(isinstance(slotids,list)):
        slots=",".join(map(str,slotids))
    else:
        slots=str(slotids)

    cursor.execute(''' Select seq+1 from sqlite_sequence where name='events' ''')    
    eventid=cursor.fetchone()[0]
    
        
    # Check if valid user ()
    # If today, check if start time later than current time
    datestatus=check_valid_date(eventdate)
    if(datestatus=='G' or datestatus=='T'):
        if(datestatus=='T' and check_valid_time(starttime)):
            
            cursor.execute(''' INSERT INTO events(pk_event_id,event_name,event_desc,event_date,start_time,end_time,user_id,venue_id,event_capacity,event_status,members_joined,slots,gender_option) VALUES(?,?,?,?,?,?,?,?,?,'A',1,?,?) ''',(eventid,eventname,eventdesc,eventdate,starttime,endtime,userid,venueid,eventcapacity,slots,genderoption))
            # Insert entry in event_members
            cursor.execute(''' INSERT INTO event_members VALUES(?,?) ''',(eventid,userid))
            # Check if entries exist in Slots
            slotspresent = check_venue_slots(venueid, eventdate)
            if ~slotspresent:
                for i in range(13):
                    cursor.execute(''' INSERT INTO slots VALUES(?+1,CAST(? as INTEGER),?,'A') ''',(i,venueid,eventdate))

            for i in slotids:
                cursor.execute(''' Update slots set availability='U' where venue_id=? and slot_id=? ''',(venueid,i))

            print("Slots booked for time:",starttime,":00  to ",endtime,":00  on date:",eventdate," for Venue ID:",venueid)
        else:
            print("Enter a valid integer hour that is greater than current time. Please retry.")
    else:
        print("The provided date has passed. Unable to book event for past date.")
    
    db.commit()

## Test Start_Event

In [357]:
eventdata=dict()
eventdata['eventname']=input("Event Name:")
eventdata['eventdesc']=input("Event Description:")
eventdata['eventdate']=input("Event Date(YYYY-MM-DD):")
eventdata['starttime']=input("Start Time (Hour in 24-hr format):")
eventdata['endtime']=input("End Time (Hour in 24-hr format):")
eventdata['venueid']=input("Venue ID:")
eventdata['eventcapacity']=input("Total Event Capacity:")
eventdata['genderoption']=input("Gender Option:")
eventdata['userid']=input("User ID:")

#eventdata = {'eventname':'Zilker-Footy', 'eventdesc': 'A friendly between UT Grads', 'eventdate': '2019-07-21', 'starttime':8, 'endtime': 10, 'venueid': 202,'eventcapacity': 4, 'genderoption': 'C', 'userid':101}

start_event(**eventdata)

Event Name:MSF
Event Description:MSF
Event Date(YYYY-MM-DD):2019-07-29
Start Time (Hour in 24-hr format):7
End Time (Hour in 24-hr format):10
Venue ID:201
Total Event Capacity:20
Gender Option:M
User ID:101

Invalid timings entered. Valid values are : 
8-20 for Start time
9-21 for End time


# 4. Display timeslot availability at a venue

In [370]:
# Display timeslot availability at a selected venue

def display_available_slots(venueid,availdate) :
        
    cursor.execute('''SELECT start_time FROM timeslots t JOIN slots s ON t.slot_id=s.slot_id WHERE s.venue_id=? AND s.date=?  AND s.availability=\'A\' ''',(venueid,availdate))
    starttimes = cursor.fetchall()
    for starttime in starttimes:
        print(str(starttime[0])+":00")

# Test timeslot availability at a venue

In [369]:
venueid=input("Enter VenueID:")
availdate=input("Enter Date to check availability(YYYY-MM-DD):")

display_available_slots(venueid,availdate)

Enter VenueID:202
Enter Date to check availability(YYYY-MM-DD):2019-07-30
[(11,), (12,), (13,), (14,), (15,), (16,), (17,), (18,), (19,), (20,)]
11:00
12:00
13:00
14:00
15:00
16:00
17:00
18:00
19:00
20:00


## 5. Display all venues where particular time slot is available

Selects all the available venues for a particular chosen time slot

In [378]:
# Selects venues where a certain time slot is available

def display_venues_for_slot(starttime, endtime, date) :

    slotids=get_slot_ids(starttime, endtime)
    
    print ("\n\nThe Venues that have %s => %s Slots available:" % (starttime, endtime))

    qmark= "?"
    if isinstance(slotids,list):
        qmarks= ",".join(qmark for slotid in slotids)
    else:
        qmarks=qmark

    selectNonEventsquery = "SELECT distinct s.venue_id, v.venue_name from slots s join venue v on v.pk_venue_id=s.venue_id where venue_id not in (SELECT distinct venue_id from slots where availability ='U' and slot_id in ({0}) and date='{1}') UNION SELECT distinct v.pk_venue_id, v.venue_name from venue v where pk_venue_id not in (SELECT venue_id from slots)".format(qmarks,date)

    cursor.execute(selectNonEventsquery, slotids)
    venuesforslot=cursor.fetchall()
        
    return venuesforslot

## Test Display_Venues_For_Slot

In [377]:
starttime=input("Start Time (Hour of the day):")
endtime=input("End Time (Hour of the day):")
date=input("Enter the date(YYYY-MM-DD):")

venues=display_venues_for_slot(int(starttime),int(endtime),date)
for t in venues:
    print(t[1])

Start Time (Hour of the day):10
End Time (Hour of the day):12
Enter the date(YYYY-MM-DD):2019-07-30
The Venues that have 10 => 12 Slots available:
Clark Field 1


# 6. List events at a venue given date/time

In [379]:
# Selects venues where a certain time slot is available

def display_venue_events(venueid,eventdate,starttime,endtime) :

    cursor.execute(''' SELECT event_name, start_time, end_time FROM events
                        WHERE venue_id=? 
                        AND event_date=? 
                        AND start_time<?
                        AND end_time>? ''',(venueid,eventdate,endtime,starttime))
    
    events=cursor.fetchall()
    for event in events:
        print("\n\nEvent Name:",event[0],"\nEvent Timing:",event[1], ": 00 to",event[2],":00\n","------------------------------------------","\n")

# Test list events at a venue given date/time

In [380]:
venueid=input("Enter VenueID of user:")
eventdate=input("Enter EventDate:")
starttime=input("Enter StartTime of event:")
endtime=input("Enter EndTime of event:")

display_venue_events(venueid,eventdate,starttime,endtime)

Enter VenueID of user:202
Enter EventDate:2019-07-30
Enter StartTime of event:9
Enter EndTime of event:15


Event Name: MSITM match 
Event Timing: 8 : 00 to 11 :00
 ------------------------------------------ 



## 7. User Joins an Event

In [384]:
# Updates tables when a user joins an event

def user_joins_event(userid, eventid) :

    #Validate userid, eventid
    cursor.execute(''' Select event_capacity, members_joined from events where pk_event_id=? ''', (eventid,))
    eventroom=cursor.fetchone()

    if eventroom[0]>eventroom[1]:
        # Update Events table
        cursor.execute(''' Update events set members_joined=members_joined+1 where pk_event_id=? ''', (eventid,))

        # Update EventMembers table
        cursor.execute(''' INSERT INTO event_members values (?,?) ''',(eventid, userid))
        print("UserID: {0} has joined event: {1}".format(userid,eventid))
    else:
        return "Event is full"
    

    db.commit()

## Test User_Joins_Event

In [383]:
userid=input("Enter UserID of user:")
eventid=input("Enter EventID joined:")

user_joins_event(userid,eventid)

Enter UserID of user:102
Enter EventID joined:301
UserID: 102 has joined event: 301


# 8. Remove an event (admin only)

In [387]:
# Removes an event

def deactivate_event(eventid,userid) :

    cursor.execute('''SELECT admin_status FROM user WHERE pk_user_id=?''',(userid,)) 
    adminstatus=cursor.fetchone()
    if adminstatus==('A',):

        cursor.execute('''UPDATE events SET event_status='U' WHERE pk_event_id=?''',(eventid,))

        cursor.execute(''' SELECT slots from events where pk_event_id= ? ''',(eventid,))
        slotids=cursor.fetchone()[0]
        slotlist=slotids.split(',')
        # Conversion to integers
        int_slotlist = [int(i) for i in slotlist] 
        
        qmark= "?"
        if isinstance(slotlist,list):
            qmarks= ",".join(qmark for slotid in slotlist)
        else:
            qmarks=qmark

        query = "UPDATE slots SET availability=\'A\' WHERE venue_id=(SELECT venue_id FROM  events WHERE pk_event_id={0}) AND date = (SELECT event_date FROM events WHERE pk_event_id={1}) AND slot_id IN ({2})".format(eventid,eventid,qmarks)
        cursor.execute(query,int_slotlist)

        cursor.execute('''DELETE FROM event_members WHERE event_id=?''',(eventid,))
        print("\nEvent with ID: {0} has been removed by User ID: {1}".format(eventid,userid))

    else:
        print("\nUser is not an admin. Only admin users are allowed to remove an event.")

    db.commit()
    

# Test remove an event (admin only)

In [391]:
eventid=input("Enter EventID joined:")
userid=input("Enter UserID of user:")

deactivate_event(eventid,userid)

Enter EventID joined:301
Enter UserID of user:101

Event with ID: 301 has been removed by User ID: 101


In [392]:
db.close()

## Clean Up SQL Scripts

In [248]:
# Import sqlite3 module
import sqlite3

# Connect to the soccerhours database
db=sqlite3.connect("/Users/subhayuchakravarty/AGIT/Github/soccerhours/soccerhours/soccerhours.db")
cursor = db.cursor()

In [288]:
cursor.execute('''DELETE FROM event_members ''')
cursor.execute('''DELETE FROM events ''')
cursor.execute('''DELETE FROM timeslots ''')
cursor.execute('''DELETE FROM slots ''')
cursor.execute('''DELETE FROM venue ''')
cursor.execute('''DELETE FROM user ''')
cursor.execute('''DELETE FROM sqlite_sequence ''')

db.commit()

In [ ]:
cursor.execute('''DROP TABLE event_members ''')
cursor.execute('''DROP TABLE events ''')
cursor.execute('''DROP TABLE timeslots ''')
cursor.execute('''DROP TABLE slots ''')
cursor.execute('''DROP TABLE venue ''')
cursor.execute('''DROP TABLE user ''')

db.commit()